In [1]:
include Math

require 'numo/narray'

def log(x)
  Numo::NMath.log(x)
end

def exp(x)
  Numo::NMath.exp(x)
end

def sin(x)
  Numo::NMath.sin(x)
end

def cos(x)
  Numo::NMath.cos(x)
end

def sqrt(x)
  Numo::NMath.sqrt(x)
end

:sqrt

# 70. データの入手・整形
文に関する極性分析の正解データを用い，以下の要領で正解データ（sentiment.txt）を作成せよ．

rt-polarity.posの各行の先頭に"+1 "という文字列を追加する（極性ラベル"+1"とスペースに続けて肯定的な文の内容が続く）
rt-polarity.negの各行の先頭に"-1 "という文字列を追加する（極性ラベル"-1"とスペースに続けて否定的な文の内容が続く）
上述1と2の内容を結合（concatenate）し，行をランダムに並び替える
sentiment.txtを作成したら，正例（肯定的な文）の数と負例（否定的な文）の数を確認せよ．

In [2]:
pos = File.open("../data/rt-polaritydata/rt-polarity.pos").read
neg = File.open("../data/rt-polaritydata/rt-polarity.pos").read

pos_tagged = pos.split("\n").map {|line| "+1 #{line}"}
neg_tagged = neg.split("\n").map {|line| "-1 #{line}"}
sen = (pos_tagged + neg_tagged).shuffle
counter = {"+1" => 0, "-1" => 0}

out = File.open("../data/sentiment.txt", "w")
sen.each do |line|
  out.puts line
end
out.close

open('../data/sentiment.txt').each_line do |line|
  counter[line[0..1]] += 1
end

counter

{"+1"=>5330, "-1"=>5330}

# 71. ストップワード
英語のストップワードのリスト（ストップリスト）を適当に作成せよ．さらに，引数に与えられた単語（文字列）がストップリストに含まれている場合は真，それ以外は偽を返す関数を実装せよ．さらに，その関数に対するテストを記述せよ．

In [3]:
word_freq = Hash.new { |h, k| h[k] = 0 }

open('../data/sentiment.txt').each_line do |line|
  line[2..-1].chomp.split.each do |word|
    word_freq[word] += 1
  end
end

# 出現頻度が高い単語上位100語をストップワードにした
# stop_list = word_freq.sort {|(_, v1), (_, v2)| v2 <=> v1}[0..100].map {|arr| arr[0]}
# stop_list += word_freq.select {|k, v| v < 5}.map {|arr| arr[0]}

stop_list = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won', 'wouldn']

def is_stopped?(word, stop_list)
  stop_list.include? word
end

# goodとかがストップワードになっちゃった...
puts is_stopped?("lion", stop_list)
puts is_stopped?("does", stop_list)
puts is_stopped?("good", stop_list)

false
true
false


# 72. 素性抽出
極性分析に有用そうな素性を各自で設計し，学習データから素性を抽出せよ．素性としては，レビューからストップワードを除去し，各単語をステミング処理したものが最低限のベースラインとなるであろう．



In [4]:
all_words = []
tf_vecs = []
labels = []

File.foreach('../data/sentiment.txt').with_index do |line, index|  
  tf_vec = Hash.new { |h, k| h[k] = 0 }  
  words = line[3..-1].chomp.split.reject {|word| is_stopped?(word, stop_list) }
  all_words += words
  
  words.each do |word|
    tf_vec[word] += 1
  end
  
  labels << line[0..1]
  tf_vecs << tf_vec  
end

all_words.uniq!
word_index = {}

all_words.each_with_index do |word, index|
  word_index.store(word, index)
end

1

1

In [76]:
x = Numo::Float64.zeros(tf_vecs.size, all_words.size)
y = Numo::Float64.zeros(tf_vecs.size, 1)
w = Numo::Float64.new(all_words.size, 1).rand

labels.each_with_index do |label, i|
  y[i] = label == '+1' ? 1.0 : 0.0 
end

tf_vecs.each_with_index do |hash, i|
  hash.each do |word, count|
    x[i, word_index[word]] = count
  end
end

1

1

In [77]:
a = Numo::Float64.new(x.shape[1], 1).rand
b = Numo::Float64.new(x.shape[1], 1).rand
w = sqrt(-2 * log(a)) * cos(2 * PI * b)

1

1

# 73. 学習
72で抽出した素性を用いて、
ロジスティック回帰モデルを学習せよ．

\begin{align}
J(\theta) = - y log(h_{\theta}(x))  - (1 - y) log(1 - h_{ \theta }(x)) \\
h_{\theta} (x) = \frac{1}{e^{-{\theta}^T x}} \\
\frac{\partial J(\theta)}{\partial x} = x^{T} (h_{\theta}(x) - y)
\end{align}

In [68]:
def h(w, x, y)
  t = x.dot(w)
  1.0 / (1.0 + exp(-t))
end

:h

In [69]:
def grad(w, x, y)
  x.transpose.dup.dot(h(w, x, y) - y)
end

:grad

In [78]:
10.times do
  w = w - 0.01 * grad(w, x, y)
end

10

In [79]:
h(w, x, y).to_a

[[6.720247180059606e-24], [1.2670904402746405e-39], [4.2491670601250386e-57], [2.898613777080052e-27], [1.0318542810997315e-38], [1.7750156116040116e-41], [6.3122453925459715e-22], [7.994028152716214e-58], [9.488820704455459e-62], [8.62361568179028e-58], [1.1495416892832417e-41], [2.4126149444502996e-67], [1.3615422599513695e-24], [1.4776796768577167e-39], [9.93007999107872e-25], [3.2544292953862298e-40], [1.1167460252661729e-21], [0.009988109162795859], [1.1075654340013505e-22], [1.236844249900121e-55], [8.701531526474558e-55], [4.574036730384208e-22], [3.804726416580007e-23], [2.4393850378142757e-62], [5.314171628358041e-22], [7.739466692936687e-32], [5.172879727681045e-20], [2.2343922324966204e-42], [4.101948669715446e-59], [1.4176408085662867e-38], [6.7989523709406955e-59], [2.5592976150195883e-21], [2.661644760289439e-54], [2.337750087495321e-38], [2.2465686823318657e-41], [8.860099175269951e-61], [4.354677947391287e-25], [2.1657436305152474e-89], [7.544527972540198e-74], [5.29797